In [1]:
import pika, os, time
import json
import ast
import numpy as np
import threading

QUEUE_NAME = 'kc.ca.queue'
EXCHANGE = 'kc.ca.exchange'

class ThreadedConsumer(threading.Thread):
  def __init__(self):
    threading.Thread.__init__(self)
    credentials = pika.PlainCredentials('rnd', 'Rnd@2143')
    parameters = pika.ConnectionParameters('172.31.34.107', 5672, '/', credentials)
    connection = pika.BlockingConnection(parameters)
    self.channel = connection.channel()
    self.channel.queue_declare(queue=QUEUE_NAME, auto_delete=False)
    self.channel.exchange_declare(exchange=EXCHANGE, durable=True, exchange_type='direct')
    self.channel.basic_qos(prefetch_count=10)
    self.channel.basic_consume(QUEUE_NAME, on_message_callback=self.callback)
    threading.Thread(target=self.channel.basic_consume(QUEUE_NAME, on_message_callback=self.callback))

  def callback(self, channel, method, properties, body):
    message = ast.literal_eval(body.decode('utf-8'))
    np.save('message.npy', message)
    
  def run(self):
    self.channel.start_consuming()

def run_consumer():
    td = ThreadedConsumer()
    td.start()
    td.join(0)

In [2]:
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, Layout, HBox, VBox
import pandas as pd
import time
import re
import os
import sys
import requests
import subprocess
import numpy as np
from json import JSONDecodeError
import sqlite3
from sqlite3 import OperationalError

data_dir = './data/experiments'
data_files = [' '] + os.listdir(data_dir)
if 'message.npy' in os.listdir():
    os.remove('message.npy')

# Widget styles
default = (data_files[0], ' ')
style = {'description_width': 'initial'}
features_layout = {'width': 'max-content','height':'200px'}

# File selection menu
file_dd=widgets.SelectMultiple(options=data_files,value=default,
    description='File:',style=style,layout=features_layout)
# Analyse button 
run_button = widgets.Button(description = "Analyse",style=style)
run_button.style.button_color = 'lightgreen'
# Metric menues
metrics_layout = {'display':'flex','width': '130px','height':'30px', 'justify_content':'flex-end'}
options = list(np.arange(1,11))
options = [str(o) for o in options]
metrics_optimize = {'min_max_tpc': ('min', 1), 'wcss': ('min', 1), 'bcss': ('max', 1), 'ch_index': ('max', 1),\
'db_index':('min', 1), 'silhouette':('max', 1), 'words_pairs': ('max', 1)}
metrics = list(metrics_optimize.keys())
metrics_menues = {}
for metric in metrics:
    menue=widgets.Dropdown(options=options,value='1',description=metric, layout=metrics_layout)
    metrics_menues[metric]=menue
# Granularity slider
granularity = widgets.IntSlider(value=100, min=2, max=1000, step=1, description='Number of Clusters',\
                                     orientation='horizontal',readout=True, readout_format='d',\
                                     style = {'description_width': 'initial'}, layout=Layout(width='400px'))
apply_granularity = widgets.ToggleButton(value=False, description='Select granularity level?',
    disabled=False, button_style='info', tooltip='Description',
    icon='check', layout=Layout(width='200px'))
# Minimal cluster size
min_cluster_menue=widgets.Dropdown(options=['0']+ options,value='4',\
                       description='Minimum number of tasks in cluster',\
                                   style = {'description_width': 'initial'},\
                                   layout=Layout(width='300px'))

# Service    
url = 'http://127.0.0.01:6002/cluster_analysis/api/v0.1/clustering'
out = widgets.Output() 
metrics_optimize = {'min_max_tpc': ('min', 1), 'wcss': ('min', 1), 'bcss': ('max', 1), 'ch_index': ('max', 1),\
'db_index':('min', 1), 'silhouette':('max', 1), 'words_pairs': ('max', 1)}
conn = sqlite3.connect('./results/CAdb')
c = conn.cursor()

def print_results(b):
    #clear_output()
    cluster_key = clusters_keys_dd.value[0]
    if cluster_key == ' ':
        print('No key selected')
    else:
        clustering_result = np.load('message.npy', allow_pickle=True)[()]
        clustering_result = list(clustering_result.values())[0]
        clusters_keys = list(clustering_result.keys())
        sep_length = len(cluster_key)+5 
        # Print results
        print(sep_length*'=')
        print('key:',cluster_key)
        print(sep_length*'-')
        names = list(clustering_result[cluster_key])
        for name in names: print(name)

def run_service(b):
    file = file_dd.value[0]
    if file == ' ':
        print('No file selected')
    else:
        config = {}
        # Experiment id
        experiment_ids = pd.read_sql_query("SELECT experiment_id from experiments", conn).astype(int)
        if len(experiment_ids) == 0: experiment_id = 1
        else: experiment_id = int(max(experiment_ids.values)[0]) + 1
        
        print('experiment_id:', experiment_id)
        config['experiment_id'] = experiment_id
        
        min_cluster_size = min_cluster_menue.value[0]
        print('min_cluster_size:', min_cluster_size)
        config['min_cluster_size'] = min_cluster_size
        
        # Metric weights
        for metric, menue in metrics_menues.items():
            config[metric] = menue.value[0]
        if apply_granularity.value:
            config['num_clusters'] = granularity.value
        data_path = os.path.join(data_dir, file)
        files = {'file': open(data_path, 'rb')}
        response = requests.post(url, files=files, data=config)
        run_consumer()
        clusters = np.load('message.npy', allow_pickle=True)[()]
        print('Best performing run id=', list(clusters.keys())[0])
        experiment_ids = pd.read_sql_query("SELECT experiment_id FROM experiments", conn).astype(int)
        current_id = int(max(experiment_ids.values)[0])
        run_cols = ['project_name', 'customer', 'run_start', 'run_end', 'duration', 'tasks_count']
        experiment_df = pd.read_sql_query("SELECT * FROM experiments \
        WHERE experiment_id={cid}".format(cid=current_id), conn).drop(run_cols, axis=1)
        display(HTML('<h1 style="color:magenta">Run Scores </h1>'))
        display(experiment_df)
        print('Clusters ready for drill down analysis')

In [14]:
# Dashboard
run_button.on_click(run_service)
display(HTML('<h1 style="color:magenta">Cluster Activities</h1>\
              <p style="color:blue">Use to following menus to submit a file for analysis:</p>\
                 <ul>\
                  <li style="color:magenta">File to analyze</li>\
                  <li style="color:magenta">Select granularity level</li>\
                  <li style="color:magenta">Set weights for validation metrics</li>\
                </ul>'))
file_box = VBox(children=[file_dd, run_button])
metrics_box = VBox(children=list(metrics_menues.values()))
config_box = VBox(children=[apply_granularity, granularity, min_cluster_menue])
HBox(children=[file_box, config_box, metrics_box])

experiment_id: 39
min_cluster_size: 0
Best performing run id= 7


,experiment_id,run_id,file_name,num_files,language_model,clustering_method,clustering_params,num_clusters,mean_duration_std,tasks_per_cluster_mean,tasks_per_cluster_median,min_tasks_per_cluster,max_tasks_per_cluster,min_max_tpc,wcss,bcss,ch_index,db_index,silhouette,words_pairs
0,38,1,CCGTD1_IPS,1,all-MiniLM-L6-v2,AgglomerativeClustering,affinity|euclidean,90,22.6,40.2,33.0,6,138,132,1534.54,1566.27,40.46,2.37,0.24,68
1,38,2,CCGTD1_IPS,1,all-MiniLM-L6-v2,AgglomerativeClustering,affinity|euclidean,180,20.57,20.1,15.0,5,83,78,1211.64,1889.19,29.95,2.06,0.3,143
2,38,3,CCGTD1_IPS,1,all-MiniLM-L6-v2,AgglomerativeClustering,affinity|euclidean,271,19.56,13.35,11.0,3,78,75,1011.87,2088.99,25.59,1.85,0.33,219
3,38,4,CCGTD1_IPS,1,all-MiniLM-L6-v2,AgglomerativeClustering,affinity|euclidean,361,18.7,10.02,8.0,2,78,76,868.39,2232.5,23.26,1.68,0.37,300
4,38,5,CCGTD1_IPS,1,all-MiniLM-L6-v2,AgglomerativeClustering,affinity|euclidean,452,17.71,8.0,6.0,2,78,76,752.83,2348.04,21.89,1.54,0.4,379
5,38,6,CCGTD1_IPS,1,all-MiniLM-L6-v2,AgglomerativeClustering,affinity|euclidean,542,17.31,6.68,5.0,2,78,76,656.18,2444.79,21.18,1.41,0.42,459
6,38,7,CCGTD1_IPS,1,all-MiniLM-L6-v2,AgglomerativeClustering,affinity|euclidean,633,16.57,5.72,4.0,1,78,77,575.17,2525.92,20.74,1.32,0.43,541
7,38,8,CCGTD1_IPS,1,all-MiniLM-L6-v2,AgglomerativeClustering,affinity|euclidean,723,15.12,5.0,4.0,1,78,77,506.47,2594.68,20.54,1.21,0.45,617


Clusters ready for drill down analysis


In [12]:
display(HTML('<h1 style="color:magenta">Cluster names for best performing run </h1>'))
clusters_keys = [' ']
if 'message.npy' in os.listdir():
    clusters = np.load('message.npy', allow_pickle=True)[()]
    clusters = list(clusters.values())[0]
    clusters_keys += list(clusters.keys())
default = (clusters_keys[0], ' ')
clusters_keys_dd=widgets.SelectMultiple(options=clusters_keys, value=default,
    description='Clusters Keys:',style=style,layout=features_layout)

# Run buttons
button = widgets.Button(description = "Activity Names",style=style)
button.style.button_color = 'lightgreen'

output = widgets.Output()
button.on_click(print_results)
HBox(children=[clusters_keys_dd, button])

key: cooling water section dpt zone middle pipe
-----------------------------------------------
DPT /Zone A (Southern Section) - Install Cooling Water Pipe (39 nos.)
DPT /Zone A (Southern Section) - Testing to Cooling Water Pipe (TBC)
DPT /Zone B (Middle Section) - Testing to Cooling Water Pipe (TBC)
DPT /Zone C (Northern Section) - Install Cooling Water Pipe
DPT /Zone C (Northern Section) - Testing to Cooling Water Pipe
key: utilities diversion shaft
------------------------------
Shaft D - Utilities Diversion
Shaft E - Utilities Diversion
key: western culvert discharge section amp zone pile curtain pipe
-----------------------------------------------------------------
Discharge Culvert /Zone A (Eastern Section) - Pipe Pile (PPS080-240, PPS330-490 excl' 1800 culvert)
Discharge Culvert /Zone  A (Eastern Section) - Grout curtain behind pile wall - excl' 1800 drainage area
Discharge Culvert /Zone A1 (Eastern Section) - Install drainage pipes
Discharge Culvert /Zone A1 (Eastern Section) -

In [5]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')